<a href="https://colab.research.google.com/github/juanfrans-courses/threads-storytelling-with-maps-and-data/blob/main/07_Advanced_Techniques/Advanced_Techniques_Geocoding_and_Extracting_Data_From_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Techniques -  Geocoding and Extracting Data from Tables

**Threads - Storytelling with Maps and Data (Spring 2025)**

* [Documentation for tabula.py](https://tabula-py.readthedocs.io/en/latest/)
* [Documentation for Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview)

## Importing data from a PDF table

### Set up

Import the usual libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import altair as alt

Install Tabula's python library

In [2]:
!pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 26.3 MB/s eta 0:00:00


Import Tabula

In [5]:
import tabula

### Import the PDF table with Tabula

In [6]:
# This can be the path to a local file or to a URL that contains the PDF
pdf_path = 'https://www.nyc.gov/assets/buildings/pdf/ll33_Data_Disclosure_2022-CBL.pdf'

In [7]:
# The function `read_pdf` returns a **list** of DataFrames (one for each page)
data_list = tabula.read_pdf(pdf_path, pages='all')
print(len(data_list), ' pdf pages where imported (each into its own DataFrame)')

697  pdf pages where imported (each into its own DataFrame)


Let's print the head of the first DataFrame to see if everything was imported correctly.

In [8]:
data_list[0].head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F
1,1000020002,10,SOUTH STREET,"1 57,426",2,D
2,1000047501,1,WATER STREET,"2 ,542,563",64,C
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B
4,1000080039,32,PEARL STREET,"4 6,724",74,B


## Geocoding addresses with the Google Maps Geocoding API

To demonstrate how to geocode addresses we will use the first DataFrame only.

In [9]:
data = data_list[0]

In [10]:
data.head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F
1,1000020002,10,SOUTH STREET,"1 57,426",2,D
2,1000047501,1,WATER STREET,"2 ,542,563",64,C
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B
4,1000080039,32,PEARL STREET,"4 6,724",74,B


### Prepare the data for geocoding

In order for the geocoding to work well, we need the addresses to be as complete and correct as possible.

First, we will use the BBL code to determine the borough and add is as a column. We will add this later to the address.

In [11]:
# Convert the BBL code to a string (it is originally read as an integer)
data['10-Digit BBL'] = data['10-Digit BBL'].astype(str)

In [12]:
# Create a new column called `boro_code` that will hold the borough code
# This comes from the first digit in the BBL number
data['boro_code'] = data['10-Digit BBL'].str[0:1]

In [13]:
data.head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade,boro_code
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F,1
1,1000020002,10,SOUTH STREET,"1 57,426",2,D,1
2,1000047501,1,WATER STREET,"2 ,542,563",64,C,1
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B,1
4,1000080039,32,PEARL STREET,"4 6,724",74,B,1


Now we create a dictionary that will hold each borough code and its corresponding borough name.

In [14]:
boro_codes = {'1': 'Manhattan', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}

In [15]:
# Use the dictionary to create a new column and "map" the codes to the names
data['borough'] = data['boro_code'].map(boro_codes)

In [16]:
data.head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade,boro_code,borough
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F,1,Manhattan
1,1000020002,10,SOUTH STREET,"1 57,426",2,D,1,Manhattan
2,1000047501,1,WATER STREET,"2 ,542,563",64,C,1,Manhattan
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B,1,Manhattan
4,1000080039,32,PEARL STREET,"4 6,724",74,B,1,Manhattan


In [17]:
# Build the address field
data['address'] = data['Street\rNumber'].astype('str') + ' ' + data['Street Name'] + ', ' + data['borough'] + ', New York, NY '

In [18]:
data.head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade,boro_code,borough,address
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F,1,Manhattan,"1 GOVERNORS ISLAND, Manhattan, New York, NY"
1,1000020002,10,SOUTH STREET,"1 57,426",2,D,1,Manhattan,"10 SOUTH STREET, Manhattan, New York, NY"
2,1000047501,1,WATER STREET,"2 ,542,563",64,C,1,Manhattan,"1 WATER STREET, Manhattan, New York, NY"
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B,1,Manhattan,"125 BROAD STREET, Manhattan, New York, NY"
4,1000080039,32,PEARL STREET,"4 6,724",74,B,1,Manhattan,"32 PEARL STREET, Manhattan, New York, NY"


### Import the geocoding libraries and authenticate your key

Remember, you need to add your geocoding api key as a "secret" through the left-hand panel

In [19]:
from google.colab import userdata
GEOCODING_API_KEY = userdata.get('GEOCODING_API_KEY')

In [20]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=dcd0c8308eaca7fcb957cca795ba100f2b9688322248227a4d413cae5970ed5b
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [21]:
import googlemaps

In [22]:
# Initiallize the API client with your key
gmaps = googlemaps.Client(key=GEOCODING_API_KEY)

### Create the main geocoding function

In [23]:
def geocode_address(address):
    '''This function takes an address and returns the latitude and longitude'''
    try:
        geocode_result = gmaps.geocode(address)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return pd.Series({'latitude': location['lat'], 'longitude': location['lng']})
        else:
            return pd.Series({'latitude': None, 'longitude': None})
    except Exception as e:
          print(f"Error geocoding {address}: {e}")
          return pd.Series({'latitude': None, 'longitude': None})

### Geocode the addresses

In [24]:
data[['latitude', 'longitude']] = data['address'].apply(geocode_address)

In [25]:
data.head()

,10-Digit BBL,Street\rNumber,Street Name,DOF Gross Square\rFootage,Energy Star 1 to 100 Score,Energy\rEfficiency\rGrade,boro_code,borough,address,latitude,longitude
0,1000010010,1,GOVERNORS ISLAND,"2 ,598,091",missing required benchmarking\rinformation,F,1,Manhattan,"1 GOVERNORS ISLAND, Manhattan, New York, NY",40.688778,-74.019103
1,1000020002,10,SOUTH STREET,"1 57,426",2,D,1,Manhattan,"10 SOUTH STREET, Manhattan, New York, NY",40.701119,-74.011879
2,1000047501,1,WATER STREET,"2 ,542,563",64,C,1,Manhattan,"1 WATER STREET, Manhattan, New York, NY",40.702125,-74.012163
3,1000057501,125,BROAD STREET,"1 ,354,691",76,B,1,Manhattan,"125 BROAD STREET, Manhattan, New York, NY",40.702373,-74.010628
4,1000080039,32,PEARL STREET,"4 6,724",74,B,1,Manhattan,"32 PEARL STREET, Manhattan, New York, NY",40.703112,-74.012281


## Test the geocoding results

To make sure the geocoding worked well, we can create a quick map and plot the locations

In [26]:
ntas = gpd.read_file('https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Neighborhood_Tabulation_Areas_2020/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson')

In [27]:
boroughs = ntas.dissolve(by='BoroCode')

In [29]:
boroughs_map = alt.Chart(boroughs[boroughs['BoroName'] == 'Manhattan']).mark_geoshape(fill=None, stroke='black').project(type='mercator')
test_map = alt.Chart(data).mark_circle(size=60, color='red').encode(
    longitude='longitude:Q',
    latitude='latitude:Q'
)
boroughs_map + test_map

alt.LayerChart(...)